# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [ ]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [ ]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

df_random_peor_caso = pd.read_csv("instancias/rand-bad/indice.csv");
df_random_repe = pd.read_csv("instancias/rand-good/indice.csv");
df_random_worst_case = pd.read_csv("instancias/worst-case/indice.csv");
df_random_best_case = pd.read_csv("instancias/best-case/indice.csv");
df_random = pd.read_csv("instancias/random/indice.csv");

La siguiente función recibe parámetros para procesar la lista de instancias particulares usando ContarPalabras.cpp.
Parametros:
- **Th_l**: Cantidad de threads lectura.
- **Th_m**: Cantidad maxima de threads a usar en paralelo.
- **List_Arch**: Lista de archivos sobre las cuales se puede buscar la maxima cantidad de apariciones. (#List>0)

In [ ]:
def correr_experimento(Th_l, Th_m, List_Arch):
    
    instancias = ["" for i in range (len(List_Arch)+3)]
    # Leo las instancias de la lista de archivos.
    for i in range(len(List_Arch)):
        instancias[i+3] = List_Arch[i]
        
    instancias[0] = "../build/ContarPalabras"
    instancias[1] = str(Th_l)
    instancias[2] = str(Th_m)
    
    # Crear proceso para ejecutar el codigo.
    #process = subprocess.Popen(['xargs', '-0', '../build/ContarPalabras'], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)
    process = subprocess.Popen(instancias, stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    #print("Esta por procesar")
#    out, err = process.communicate('\0'.join(instancias))
    # Correr experimento.
    exit_code = process.wait()
    
    # Verificar que el proceso no fallo.
    if exit_code != 0: print("FATAL-ERROR")#raise(F"Hubo un error en la experimentacion para el algoritmo con la instancia.")
    
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    res = process.stderr.read()
    
    # Leemos la salida
    # tiempo_thread1 ... tiempo_threadN \n
    # tiempo_hashmap tiempo_maximo
    
    tiempos = res.split("\n")
    
    tiempos_threads = tiempos[0].split(" ")
    tiempos_importantes = tiempos[1].split(" ")
    
    tiempos_threads = [(float(x) / 1e6) for x in tiempos_threads]
    
    tiempo_hashmap = float(tiempos_importantes[0]) / 1e6
    tiempo_maximo = float(tiempos_importantes[1]) / 1e6
        
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return [tiempo_hashmap, tiempo_maximo, tiempos_threads]

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [ ]:
experimentos = [];

## Experimento 1
Ejecución peorcaso con cantidad de threads variable

In [ ]:
fila_n = df_random_worst_case.iloc[0]
archivos = [fila_n["archivo"]]

for cantThreads in range(1, 26 + 1):
    experimentos.append([1, cantThreads, archivos, "worst-case"])

## Experimento 2
Ejecución mejorcaso con cantidad de threads variable

In [ ]:
fila_n = df_random_best_case.iloc[0]
archivos = [fila_n["archivo"]]

for cantThreads in range(1, 26 + 1):
    experimentos.append([4, cantThreads, archivos, "best-case"])

## Experimento 3
Ejecución random con cantidad de threads variable

In [ ]:
fila_n = df_random.iloc[0]
archivos = [fila_n["archivo"]]

for cantThreads in range(1, 26 + 1):
    experimentos.append([1, cantThreads, archivos, "random"])

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [ ]:
columnas = ["metodo", "cant_threads_archivos", "cant_threads_maximo", "tiempo_hashmap", "tiempo_maximo", "tiempos_threads"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    tiempos_threads = []
    for i in range(0, T):
        data = correr_experimento(experimento[0], experimento[1], experimento[2])
        tiempos.append([data[0], data[1]])
        tiempos_threads.append(data[2])
        
    tiempo_threads = np.median(tiempos_threads, axis=0)
    tiempo = np.median(tiempos, axis=0)
    
    filas.append([experimento[3], experimento[0], experimento[1], tiempo[0], tiempo[1], tiempo_threads])
    
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);